In [1]:

%pprint

Pretty printing has been turned OFF


In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

%%time
from scrape_utils import WebScrapingUtilities
%run ../load_magic/storage.py
from html_analysis import HeaderAnalysis
from cypher_utils import CypherUtilities

wsu = WebScrapingUtilities()
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']
s = Storage()
ha = HeaderAnalysis()
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

CPU times: total: 2.47 s
Wall time: 3.41 s


In [18]:

HEADER_PATTERN_DICT = s.load_object('HEADER_PATTERN_DICT')
list(HEADER_PATTERN_DICT.values())[0][0]

{'initial_tag': 'h2', 'is_header': True, 'is_task_scope': True, 'is_minimum_qualification': False, 'is_preferred_qualification': False, 'is_legal_notification': False, 'is_job_title': False, 'is_office_location': False, 'is_job_duration': False, 'is_supplemental_pay': False, 'is_educational_requirement': False, 'is_interview_procedure': False, 'is_corporate_scope': False, 'is_posting_date': False, 'is_other': False, 'child_str': '<h2 class="jobsearch-JobDescriptionSection-jobDescriptionTitle icl-u-xs-my--md" id="jobDescriptionTitle">Full Job Description</h2>'}

In [4]:

from html_analysis import ElementAnalysis

file_name = '9dc7201db74605e8_Data_Business_Analyst_Remote_Indeed_com.html'
child_strs_list = ha.get_child_strs_from_file(file_name)
ea = ElementAnalysis(ha=ha, hc=None)
ea.find_basic_quals_section(child_strs_list, verbose=True)


            MATCH (pos:PartsOfSpeech)-[r:SUMMARIZES]->(np:NavigableParents)
            RETURN
                np.navigable_parent AS navigable_parent,
                pos.pos_symbol AS pos_symbol;
pos_df.columns = Index(['navigable_parent', 'pos_symbol'], dtype='object')
[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

([('O', 15), ('H-TS', 1), ('O', 15), ('H-RQ', 1), ('O', 13), ('H-SP', 1), ('O', 29)], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'H-TS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'H-RQ', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'H-SP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])

In [13]:

cypher_str = '''
    MATCH (pos:PartsOfSpeech)
    UNWIND pos.pos_symbol as s
    RETURN count(DISTINCT s) AS num_topics;'''
row_objs_list = cu.get_execution_results(cypher_str, verbose=True)
row_objs_list[0]['num_topics']


    MATCH (pos:PartsOfSpeech)
    UNWIND pos.pos_symbol as s
    RETURN count(DISTINCT s) AS num_topics;


26

In [5]:

ea.display_basic_requirements(child_strs_list)

In [ ]:

from html_analysis import LrUtilities

lu = LrUtilities(ha=ha, hc=None, cu=cu, verbose=False)
predict_percent_fit = lu.build_lr_predict_percent_is_header()

In [23]:

from sklearn.feature_extraction.text import CountVectorizer

if s.pickle_exists('CS_CV'):
    lu.CS_CV = s.load_object('CS_CV')
else:
    lu.CS_CV = CountVectorizer(
        analyzer='word',
        binary=False,
        decode_error='strict',
        lowercase=False,
        max_df=1.0,
        max_features=None,
        min_df=0.0,
        ngram_range=(1,5),
        stop_words=None,
        strip_accents='ascii',
        tokenizer=ha.html_regex_tokenizer)
    s.store_objects(CS_CV=lu.CS_CV)

In [25]:

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import LogisticRegression

# Re-transform the bag-of-words and tf-idf from the new manual scores
rows_list = [{'navigable_parent': navigable_parent,
              'is_header': is_header} for navigable_parent, is_header in ha.NAVIGABLE_PARENT_IS_HEADER_DICT.items()]
child_str_df = pd.DataFrame(rows_list)
child_str_df.is_header = child_str_df.is_header.astype('bool')

# The shape of the Bag-of-words count vector here should be n sentences * m unique words
sents_list = child_str_df.navigable_parent.tolist()
bow_matrix = lu.CS_CV.fit_transform(sents_list)

# Tf-idf must get from Bag-of-words first
tfidf_matrix = lu.TT.fit_transform(bow_matrix)

In [35]:

# Re-train the classifier
try:
    import gc

    gc.collect()
    X = tfidf_matrix.toarray()
except Exception as e:
    print(f'{e.__class__} error: {str(e).strip()}')
    X = tfidf_matrix

<class 'numpy.core._exceptions._ArrayMemoryError'> error: Unable to allocate 62.0 GiB for an array with shape (11712, 710081) and data type float64


In [19]:

if s.pickle_exists('CHILD_STR_CLF'):
    lu.CHILD_STR_CLF = s.load_object('CHILD_STR_CLF')
else:
    lu.CHILD_STR_CLF = LogisticRegression(
        C=375.0,
        class_weight='balanced',
        dual=False,
        fit_intercept=True,
        intercept_scaling=1,
        l1_ratio=None,
        max_iter=1000,
        multi_class='auto',
        n_jobs=None,
        penalty='l1',
        random_state=None,
        solver='liblinear',
        tol=0.0001,
        verbose=0,
        warm_start=False)
    s.store_objects(CHILD_STR_CLF=lu.CHILD_STR_CLF)

In [26]:

y = child_str_df.is_header.to_numpy()
lu.CHILD_STR_CLF.fit(tfidf_matrix, y)

LogisticRegression(C=10.0, class_weight='balanced', max_iter=1000, penalty='l1',
                   solver='liblinear')

In [27]:

s.store_objects(CHILD_STR_CLF=lu.CHILD_STR_CLF)

Pickling to C:\Users\daveb\OneDrive\Documents\GitHub\job-hunting\saves\pkl\CHILD_STR_CLF.pkl


In [28]:

# Re-calibrate the inference engine
cv = CountVectorizer(vocabulary=lu.CS_CV.vocabulary_)
cv._validate_vocabulary()

def predict_percent_fit(navigable_parent):
    X_test = lu.TT.transform(cv.transform([navigable_parent])).toarray()
    y_predict_proba = lu.CHILD_STR_CLF.predict_proba(X_test)[0][1]

    return y_predict_proba

In [31]:

child_str_df['predict_percent_fit'] = child_str_df.navigable_parent.map(lambda x: predict_percent_fit(x))

In [34]:

mask_series = child_str_df.is_header
max_false_percent_fit = child_str_df[~mask_series].predict_percent_fit.max()
print(max_false_percent_fit)
display(child_str_df[~mask_series].sort_values('predict_percent_fit', ascending=False))
min_true_percent_fit = child_str_df[mask_series].predict_percent_fit.min()
print(min_true_percent_fit)
display(child_str_df[mask_series].sort_values('predict_percent_fit', ascending=True))

0.9999999999944673


,navigable_parent,is_header,predict_percent_fit
1498,<b>will not</b>,False,1.000000e+00
648,<b>Great analytics</b>,False,1.000000e+00
10937,"<li>Telerik &amp; Kendo UI Tools, NodeJS, Angu...",False,1.000000e+00
138,. See below for more information.,False,1.000000e+00
979,<b>Python &amp; AIMMS;</b>,False,1.000000e+00
...,...,...,...
1487,<b>or</b>,False,4.271966e-18
863,<b>OR</b>,False,4.271966e-18
249,<b>5+ years</b>,False,4.162431e-21
53,*: 3-6 years,False,4.162431e-21


3.0143009290415106e-07


,navigable_parent,is_header,predict_percent_fit
250,<b>6 years of analytics experience:</b>,True,3.014301e-07
380,<b>Big Data:</b>,True,2.746350e-06
409,<b>Careers with Optum.</b>,True,6.361448e-06
9574,Collaborate with the:,True,2.182146e-05
480,<b>Data storyteller:</b>,True,7.441793e-05
...,...,...,...
1432,<b>You will:</b>,True,1.000000e+00
1435,<b>You:</b>,True,1.000000e+00
349,<b>At Compass You Will:</b>,True,1.000000e+00
1422,<b>You Will:</b>,True,1.000000e+00
